In [42]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
import sklearn

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

togeth = pd.read_csv('C:/Users/jsult/Desktop/spacetitanic3/FINALtransferred/certainty_imputed_data',low_memory = False)
origtrain = pd.read_csv('C:/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory = False)

togeth = togeth.drop(columns = ['Unnamed: 0','VRDeck_Family.1',
       'Spa_Family.1', 'RoomService_Family.1', 'FoodCourt_Family.1',
       'ShoppingMall_Family.1'])

origcolumns = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

twonas = list(togeth[togeth[origcolumns].isna().sum(axis = 1) > 1].PassengerId)



def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)
     

In [43]:
cat  = ['HomePlanet','CryoSleep', 'Cabin', 'Destination', 'VIP','Name', 'Transported', 'train','FirstName',
       'LastName', 'CabinSide', 'Under13', 'Under18', 'Under19']
cont = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Group', 'GroupNumber','CabinNum', 'TotalAmmenities',
       'TotalCabinAmmenities', 'TotalSpending', 'TotalCabinSpending',
       'TotalCabinRoomService', 'VRDeck_Family', 'Spa_Family',
       'RoomService_Family', 'FoodCourt_Family', 'ShoppingMall_Family',
       'TotalFamilyAmmenities', 'TotalFamilySpending', 'GroupSize', 'FamilySize', 'CabinSize',]

cabinorder = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
togeth['CabinDeck'] = pd.Categorical(togeth['CabinDeck'], categories=cabinorder, ordered=True)
#agebin as float?

for item in cat:
    togeth[item] = togeth[item].astype('category')

for item in cont:
    togeth[item] = togeth[item].astype('float')


In [44]:
traindf = togeth[togeth.train == True]
testdf = togeth[togeth.train == False]
testdf = testdf.drop(columns = 'Transported')

In [45]:
goodtest = list(testdf.dropna(subset = origcolumns).index)
badtest = list(set(testdf.index).difference(set(goodtest)))

In [41]:
testdf.loc[badtest]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,train,Group,GroupNumber,FirstName,LastName,CabinDeck,CabinSide,CabinNum,TotalAmmenities,TotalCabinAmmenities,TotalSpending,TotalCabinSpending,TotalCabinRoomService,VRDeck_Family,Spa_Family,RoomService_Family,FoodCourt_Family,ShoppingMall_Family,TotalFamilyAmmenities,TotalFamilySpending,GroupSize,FamilySize,CabinSize,SoloCabin,Agebin,Under13,Under18,Under19
4098,2932_01,Europa,True,B/109/S,NaN,45.0,False,0.0,0.0,0.0,0.0,0.0,Tarain Tachor,False,2932.0,1.0,Tarain,Tachor,B,S,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,True,"(38.0, 79.0]",False,False,False
12293,8804_02,Mars,True,F/1814/P,NaN,63.0,False,0.0,0.0,0.0,0.0,0.0,Gra Sunie,False,8804.0,2.0,Gra,Sunie,F,P,1814.0,0.0,1965.0,0.0,2997.0,1032.0,7.0,1156.0,1032.0,78.0,724.0,3930.0,5994.0,5.0,3.0,5.0,False,"(38.0, 79.0]",False,False,False
10246,7318_01,Mars,False,F/1407/S,PSO J318.5-22,29.0,False,1061.0,7.0,312.0,0.0,11.0,NaN,False,7318.0,1.0,NaN,NaN,F,S,1407.0,330.0,330.0,1391.0,1391.0,1061.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,1.0,True,"(27.0, 38.0]",False,False,False
10247,7319_01,Earth,False,G/1176/P,PSO J318.5-22,13.0,False,33.0,28.0,685.0,27.0,NaN,Phia Thonyderson,False,7319.0,1.0,Phia,Thonyderson,G,P,1176.0,740.0,1561.0,773.0,1607.0,46.0,6.0,35.0,46.0,1566.0,687.0,4588.0,4680.0,3.0,3.0,2.0,False,"(-0.001, 19.0]",False,True,True
4103,2937_01,Earth,False,F/614/P,TRAPPIST-1e,22.0,False,0.0,1.0,NaN,237.0,620.0,Gerly Valey,False,2937.0,1.0,Gerly,Valey,F,P,614.0,858.0,858.0,858.0,858.0,0.0,620.0,237.0,0.0,1.0,0.0,1716.0,1716.0,1.0,1.0,1.0,True,"(19.0, 27.0]",False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12266,8793_01,Europa,False,C/289/P,55 Cancri e,18.0,False,2304.0,207.0,NaN,71.0,2724.0,Alsham Unreezy,False,8793.0,1.0,Alsham,Unreezy,C,P,289.0,3002.0,8362.0,5306.0,10666.0,2304.0,2724.0,71.0,2304.0,207.0,0.0,6004.0,10612.0,5.0,3.0,5.0,False,"(-0.001, 19.0]",False,False,True
4076,2916_01,Mars,True,F/604/P,TRAPPIST-1e,NaN,False,0.0,0.0,0.0,0.0,0.0,Mars Ickes,False,2916.0,1.0,Mars,Ickes,F,P,604.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,True,NaN,NaN,NaN,NaN
12269,8793_04,Europa,False,C/289/P,TRAPPIST-1e,52.0,NaN,0.0,713.0,0.0,4331.0,316.0,Wasaleh Oirmplathy,False,8793.0,4.0,Wasaleh,Oirmplathy,C,P,289.0,5360.0,8362.0,5360.0,10666.0,2304.0,316.0,4331.0,0.0,713.0,0.0,10720.0,10720.0,5.0,2.0,5.0,False,"(38.0, 79.0]",False,False,False
12284,8800_01,Earth,True,G/1423/P,TRAPPIST-1e,3.0,False,0.0,0.0,0.0,0.0,0.0,NaN,False,8800.0,1.0,NaN,NaN,G,P,1423.0,0.0,1766.0,0.0,1779.0,13.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0,NaN,5.0,False,"(-0.001, 19.0]",True,True,True


In [35]:
testdf.dropna(subset = origcolumns).isna().sum()

PassengerId               0
HomePlanet                0
CryoSleep                 0
Cabin                     0
Destination               0
Age                       0
VIP                       0
RoomService               0
FoodCourt                 0
ShoppingMall              0
Spa                       0
VRDeck                    0
Name                     85
train                     0
Group                     0
GroupNumber               0
FirstName                85
LastName                 85
CabinDeck                 0
CabinSide                 0
CabinNum                  0
TotalAmmenities           0
TotalCabinAmmenities      0
TotalSpending             0
TotalCabinSpending        0
TotalCabinRoomService     0
VRDeck_Family            85
Spa_Family               85
RoomService_Family       85
FoodCourt_Family         85
ShoppingMall_Family      85
TotalFamilyAmmenities     0
TotalFamilySpending       0
GroupSize                 0
FamilySize               85
CabinSize           

In [6]:
traindf.dropna()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,train,Group,GroupNumber,FirstName,LastName,CabinDeck,CabinSide,CabinNum,TotalAmmenities,TotalCabinAmmenities,TotalSpending,TotalCabinSpending,TotalCabinRoomService,VRDeck_Family,Spa_Family,RoomService_Family,FoodCourt_Family,ShoppingMall_Family,TotalFamilyAmmenities,TotalFamilySpending,GroupSize,FamilySize,CabinSize,SoloCabin,Agebin,Under13,Under18,Under19
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,True,1.0,1.0,Maham,Ofracculy,B,P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,True,"(38.0, 79.0]",False,False,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,True,2.0,1.0,Juanna,Vines,F,S,0.0,627.0,627.0,736.0,736.0,109.0,44.0,549.0,109.0,9.0,25.0,1254.0,1472.0,1.0,1.0,1.0,True,"(19.0, 27.0]",False,False,False
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,True,3.0,1.0,Altark,Susent,A,S,0.0,10340.0,15516.0,10383.0,15559.0,43.0,242.0,10044.0,43.0,4859.0,371.0,31032.0,31118.0,2.0,2.0,2.0,False,"(38.0, 79.0]",False,False,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,True,3.0,2.0,Solam,Susent,A,S,0.0,5176.0,15516.0,5176.0,15559.0,43.0,242.0,10044.0,43.0,4859.0,371.0,31032.0,31118.0,2.0,2.0,2.0,False,"(27.0, 38.0]",False,False,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,True,4.0,1.0,Willy,Santantines,F,S,1.0,788.0,788.0,1091.0,1091.0,303.0,2.0,565.0,303.0,70.0,151.0,1576.0,2182.0,1.0,1.0,1.0,True,"(-0.001, 19.0]",False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12964,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,True,9276.0,1.0,Gravior,Noxnuther,A,P,98.0,8536.0,8536.0,8536.0,8536.0,0.0,74.0,1643.0,0.0,6819.0,0.0,17072.0,17072.0,1.0,1.0,1.0,True,"(38.0, 79.0]",False,False,False
12966,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,True,9278.0,1.0,Kurta,Mondalley,G,S,1499.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,True,"(-0.001, 19.0]",False,False,True
12967,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,True,9279.0,1.0,Fayey,Connon,G,S,1500.0,1873.0,1873.0,1873.0,1873.0,0.0,0.0,1.0,0.0,0.0,1872.0,3746.0,3746.0,1.0,1.0,1.0,True,"(19.0, 27.0]",False,False,False
12968,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,True,9280.0,1.0,Celeon,Hontichre,E,S,608.0,4637.0,9337.0,4637.0,9463.0,126.0,3247.0,353.0,126.0,5737.0,0.0,18674.0,18926.0,2.0,2.0,2.0,False,"(27.0, 38.0]",False,False,False


In [ ]:
splits = RandomSplitter(seed = 1)(range_of(traindf))
dep_var = 'Transported'
procs = [Categorify,FillMissing,Normalize]
cont,cat = cont_cat_split(traindf, 1, dep_var=dep_var)
to = TabularPandas(traindf, procs, cat, cont, y_names=dep_var,splits = splits)

xs,y = to.train.xs,to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [ ]:
m = DecisionTreeRegressor(max_leaf_nodes=20)
m.fit(xs, y)

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(m,
                   feature_names= traindf.columns,
                   filled=True)

In [ ]:
((m.predict(valid_xs) > 0.5) == valid_y).sum() / len(valid_y)

In [ ]:
m = RandomForestClassifier(n_estimators= 100, max_leaf_nodes= 50)
m.fit(xs, y)
preds = m.predict(valid_xs)
print(mean_absolute_error(valid_y, preds))
(preds == valid_y).sum() /len(valid_y)

In [ ]:

testtran = [True if x == 1 else False for x in list(m.predict(to_new.xs))]
testdf['Transported'] = testtran
ret = testdf[['PassengerId','Transported']]

ret.to_csv('submit.csv',index=False)